In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
URL_sea = 'https://www.amazon.co.uk/s?k=hard+drive&i=computers&rh=n%3A340831031%2Cp_89%3ASeagate&s=review-rank&dc&ds=v1%3A6SXEIuGkAa4qAp9wc0q0WWpwv7J6X5ClyHcnKyo15ds&crid=11I57DU1Q40BL&qid=1709561950&rnid=1632651031&sprefix=hard+driv%2Caps%2C205&ref=sr_st_review-rank'
URL_west = 'https://www.amazon.co.uk/s?k=hard+drive&i=computers&rh=n%3A340831031%2Cp_89%3AWestern+Digital&s=review-rank&dc&crid=11I57DU1Q40BL&qid=1709562162&rnid=1632651031&sprefix=hard+driv%2Caps%2C205&ref=sr_nr_p_89_2&ds=v1%3A5dK5J9Pb%2FoivfgH9WeLD2TaqojHqEqgkjIRj1SupOnc'
URL_sam = 'https://www.amazon.co.uk/s?k=hard+drive&i=computers&rh=n%3A340831031%2Cp_89%3ASamsung&s=review-rank&dc&crid=11I57DU1Q40BL&qid=1709562209&rnid=1632651031&sprefix=hard+driv%2Caps%2C205&ref=sr_nr_p_89_2&ds=v1%3ABwoBkg2JqfX1oDjRFD01KrMrPG%2B%2ByLRuvFvOIfW8awo'
URL_san = 'https://www.amazon.co.uk/s?k=hard+drive&i=computers&rh=n%3A340831031%2Cp_89%3ASanDisk&s=review-rank&dc&crid=11I57DU1Q40BL&qid=1709562240&rnid=1632651031&sprefix=hard+driv%2Caps%2C205&ref=sr_nr_p_89_2&ds=v1%3ABfLP18lp%2F%2FUyQ61TyfXFBZ8MR1eKrMSiB8pQZTMy%2F0w'
URL_cru = 'https://www.amazon.co.uk/s?k=hard+drive&i=computers&rh=n%3A340831031%2Cp_89%3ACrucial&dc&ds=v1%3A3tQlFt3qboE0VHotZfUKoiCdmhUJNOnNz9%2Bw1ZjEfek&crid=13VBQIRQGNOOE&qid=1709652649&rnid=1632651031&sprefix=hard+drive%2Ccomputers%2C224&ref=sr_nr_p_89_16'

In [11]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
            'Accept-Language':'en-US,en:q=0.5'})

In [12]:
webpage_sea = requests.get(URL_sea,headers=HEADERS)
webpage_west = requests.get(URL_west,headers=HEADERS)
webpage_sam = requests.get(URL_sam,headers=HEADERS)
webpage_san = requests.get(URL_san,headers=HEADERS)
webpage_cru = requests.get(URL_cru,headers=HEADERS)

In [13]:
soup_sea = BeautifulSoup(webpage_sea.content, 'html.parser')
soup_west = BeautifulSoup(webpage_west.content, 'html.parser')
soup_sam = BeautifulSoup(webpage_sam.content, 'html.parser')
soup_san = BeautifulSoup(webpage_san.content, 'html.parser')
soup_cru = BeautifulSoup(webpage_cru.content, 'html.parser')

In [14]:
links_sea = soup_sea.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
links_west = soup_west.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
links_sam = soup_sam.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
links_san = soup_san.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
links_cru = soup_cru.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [ ]:
link_set = [links_sea, links_west,links_sam,links_san,links_cru]
df_name = ['df_sea','df_west','df_sam','df_san','df_cru']
for i in link_set:
  column_list = ['title','price','rating','rating_no','capacity','link']
  df = pd.DataFrame(columns=column_list)
  for link in i:
    link = str(link.get('href'))
    index = link.find('https://www.amazon.co.uk')
    link = link[:index]
    products = 'https://www.amazon.co.uk' + link
    try:
      new_page = requests.get(products,headers=HEADERS)
      new_soup = BeautifulSoup(new_page.content, 'html.parser')
      title = new_soup.find('span',attrs={'id':'productTitle'}).text.strip()
      price = new_soup.find('span',attrs={'class':'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).text.strip().replace('£','')
      rating = new_soup.find('span',attrs={'class':'a-icon-alt'}).text.strip()
      rating_no = new_soup.find('span',attrs={'id':'acrCustomerReviewText'}).text.strip()
      index = title.find('TB')
      cap = title[index-2:index+2].replace(' ','').replace('.','').replace(',','')
      if index == -1:
        index = title.find('GB')
        cap = title[index-4:index+2].replace(' ','').replace('.','').replace(',','')
    except Exception:
      pass
    finally:
      entry = [title,price,rating,rating_no,cap,products]
      df.loc[len(df)] = entry
  df = df.drop_duplicates(subset=['price','rating','rating_no'])
  print(df)
  num = link_set.index(i)
  name = df_name[num]
  df.to_csv(f'{name}.csv',index = False)